In [1]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt

from utils import *
from dataset import *

import pickle as pkl
data_root = 'data_root/3DGraspAff'
train_or_test = 'test' # 'train' or 'test'
aff_path = opj(data_root, 'anntd_remapped_full_shape_'+train_or_test+'_data.pkl')
with open(aff_path, 'rb') as f:
    aff_dataset = pkl.load(f)
print("Loaded dataset with {} objects".format(aff_dataset['num_objects']))
print("Num of semantic classes: ", len(aff_dataset["semantic_classes"]))
print("Semantic classes: ", aff_dataset["semantic_classes"])
print("Num of affordances: ", len(aff_dataset["affordance_classes"]))
print("All affordances: ", aff_dataset["affordance_classes"])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.
Loaded dataset with 152 objects
Num of semantic classes:  7
Semantic classes:  ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
Num of affordances:  7
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']


In [2]:
# Save the meshes needed for the remapped dataset
from utils.partnet_utils import class_to_shapenet_dir, shapeid_to_partnet_dir
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
SHAPENET_PATH = '/home/sjauhri/IAS_WS/ShapeNetCore.v2'
# PARTNET_PATH  = '/home/sjauhri/IAS_WS/PartNet/data_v0'
PARTNET_PATH  = '/home/sjauhri/IAS_WS/potato-net/affordance/3DAffordanceNet/data_root/3DGraspAff/obj_meshes/'+train_or_test+'/PartNet_full'

SHAPENET_SAVE_PATH = data_root + '/obj_meshes/'+train_or_test+'/ShapeNet'
# PARTNET_SAVE_PATH  = data_root + '/obj_meshes/'+train_or_test+'/PartNet_full'
PARTNET_SAVE_PATH  = data_root + '/obj_meshes/'+train_or_test+'/PartNet'

# scaling_parameters
shapenet_scalings = {
    'Knife': 0.17,
    'Bag': 0.165,
    'Bottle': 0.17,
    # 'Scissors': 0.165, # normally no scissors
    'Mug': 0.11,
    'Earphone': 0.15,
    'Hat': 0.165,
    }
partnet_scalings = {
    # 'Knife': 0.165, # normally no knives
    'Bag': 0.06,
    # 'Bottle': 0.165, # normally no bottles
    'Scissors': 0.065,
    # 'Mug': 0.11, # normally no mugs
    'Earphone': 0.065,
    'Hat': 0.065,
}

# loop through the dataset
for sem_classs in aff_dataset['semantic_classes']:
    shape_ids = list(aff_dataset['data'][sem_classs].keys())
    start_id = 0
    end_id =  len(shape_ids) # min(200, len(shape_ids))
    # rand_id = random.randint(start_id, end_id)
    # for shape_id in shape_ids[rand_id:rand_id+1]:
    for shape_id in shape_ids[start_id:end_id]:
        print("Processing shape_id: ", shape_id, " for class: ", sem_classs)
        shapeid_path = opj(SHAPENET_PATH, class_to_shapenet_dir[sem_classs.lower()],
                        shape_id)
        obj_path = opj(shapeid_path, 'models', 'model_normalized.obj')
        # check if obj_path exists
        if os.path.exists(obj_path):
            # try load with open3d
            obj_mesh = o3d.io.read_triangle_mesh(obj_path)
            # if successful, just copy everything to the save path
            if not os.path.exists(opj(SHAPENET_SAVE_PATH, shape_id)):
                os.makedirs(opj(SHAPENET_SAVE_PATH, shape_id))
            os.system('cp -r {} {}'.format(shapeid_path, SHAPENET_SAVE_PATH))

            # Scale the stored pointcloud to match the original shapenet mesh
            datapoint = aff_dataset['data'][sem_classs][shape_id]
            pcl = datapoint['coordinates']
            orig_points = obj_mesh.sample_points_uniformly(number_of_points=8000).points
            # get the center of the mesh
            orig_center = np.mean(orig_points, axis=0)
            # get the maximum extent of the mesh
            centered_points = orig_points - orig_center # center the points
            orig_max_dist = np.max(np.sqrt(np.sum(centered_points**2, axis=1)))
            # correct the pcl
            corrected_pcl = pcl
            # # flip along the z axis??
            corrected_pcl[:,2] = -corrected_pcl[:,2]
            # # flip along the x axis??
            corrected_pcl[:,0] = -corrected_pcl[:,0]
            # scale the point cloud
            corrected_pcl *= orig_max_dist
            # recenter the point cloud
            corrected_pcl += orig_center

            # scale the mesh and the point cloud and save
            vertices = np.asarray(obj_mesh.vertices)
            vertices *= shapenet_scalings[sem_classs]
            corrected_pcl *= shapenet_scalings[sem_classs]
            # DEBUG.....
            # break
            # .....DEBUG
            # save mesh
            o3d.io.write_triangle_mesh(opj(SHAPENET_SAVE_PATH, shape_id, 'models', 'model_normalized.obj'), obj_mesh)
        else:
            print("ShapeNet category not found, try PartNet?")
            partnet_anno_id = shapeid_to_partnet_dir[shape_id]
            
            mesh_path = opj(PARTNET_PATH, partnet_anno_id, 'model_normalized.obj')
            obj_mesh = o3d.io.read_triangle_mesh(mesh_path)
            # scale the mesh and the point cloud and save
            datapoint = aff_dataset['data'][sem_classs][shape_id]
            pcl = datapoint['coordinates']
            vertices = np.asarray(obj_mesh.vertices)
            vertices *= partnet_scalings[sem_classs]
            corrected_pcl = pcl
            corrected_pcl *= partnet_scalings[sem_classs]

            # if using original PartNet meshes
            # meshes_path = opj(PARTNET_PATH, partnet_anno_id, 'objs')
            # # loop through all meshes in the folder and add to open3d object
            # obj_mesh = o3d.geometry.TriangleMesh()
            # for mesh in os.listdir(meshes_path):
            #     # optional: only load files with 'original' in the name
            #     mesh_path = opj(meshes_path, mesh)
            #     mesh = o3d.io.read_triangle_mesh(mesh_path)
            #     obj_mesh += mesh
            
            # save
            if not os.path.exists(opj(PARTNET_SAVE_PATH, partnet_anno_id)):
                os.makedirs(opj(PARTNET_SAVE_PATH, partnet_anno_id))
            save_path = opj(PARTNET_SAVE_PATH, partnet_anno_id, 'model_normalized.obj')
            o3d.io.write_triangle_mesh(save_path, obj_mesh, write_triangle_uvs=True)
            # DEBUG.....
            # break
            # .....DEBUG

Processing shape_id:  3ef87a241213eb129f30f02514717f00  for class:  Knife
Processing shape_id:  b8fe56eeed3951392f98be9d5bb04d17  for class:  Knife
Processing shape_id:  57795189a634e21fa0f8e9a2d8ead2d5  for class:  Knife
Processing shape_id:  5391a808f36b46971638241f3ad71c10  for class:  Knife
Processing shape_id:  13bf5728b1f3b6cfadd1691b2083e9e7  for class:  Knife
Processing shape_id:  73dd7fb60dd52b85ed4efc1a1dc582cb  for class:  Knife
Processing shape_id:  854e7bb73afaff7591ea3afb2749822f  for class:  Knife
Processing shape_id:  76d4739797f4cec15d3426ee101e9f8e  for class:  Knife
Processing shape_id:  a9a091c148dcd07a156edd8a0a2b1378  for class:  Knife
Processing shape_id:  4bbde8accc3bbfb87f277d7152b6b3ed  for class:  Knife
Processing shape_id:  ee7ced71701c3e88b9e3f0d9deebf307  for class:  Knife
Processing shape_id:  ced1a454c0e5fadd567f010bb1e0a2a4  for class:  Knife
Processing shape_id:  6ca0db952c7e2169e2bb19fbad3e4596  for class:  Knife
Processing shape_id:  a73a2d9584b2cbc8

In [3]:
# # DEBUG.....
# # print min, max xyz of the corrected pcl
# print("Min xyz: ", np.min(corrected_pcl, axis=0))
# print("Max xyz: ", np.max(corrected_pcl, axis=0))
# soccer_ball = o3d.io.read_triangle_mesh('/home/sjauhri/IAS_WS/potato-net/GIGA-TSDF/GIGA-6DoF/data/urdfs/pile/test/soccer_ball_poisson_000_visual.obj')
# # paint
# soccer_ball.paint_uniform_color([1, 0, 0])
# draw([obj_mesh, o3d.geometry.PointCloud(o3d.utility.Vector3dVector(corrected_pcl)), soccer_ball])

In [4]:
# Save updated dataset
new_aff_path = opj(data_root, 'scaled_anntd_remapped_full_shape_'+train_or_test+'_data.pkl')
with open(new_aff_path, 'wb') as f:
    pkl.dump(aff_dataset, f)